In [1]:
import html
import json
import os
import time

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# Configuration des headers pour simuler un navigateur
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'
}

In [3]:
# Fonction pour rechercher des produits
def recherche_produit(data, products_list):
    if isinstance(data, dict):
        # Si c'est un produit, l'ajouter
        if 'bestprice' in data and 'title' in data:
            produit = {
                'titre': data.get('title', 'Titre non disponible'),
                'prix_solde': data.get('bestprice'),
                'prix_avant_reduction': data.get('oldprice'),
                'note': float(data.get('rate', 0)) if data.get('rate') else None,
                'lien_image': data.get('preview', '')
            }
            products_list.append(produit)

        # Continuer la recherche dans les valeurs
        for value in data.values():
            recherche_produit(value, products_list)

    elif isinstance(data, list):
        # Parcourir chaque élément de la liste
        for item in data:
            recherche_produit(item, products_list)

In [4]:
# Fonction pour extraire le contenu d'une page web
def extrait_contenu_page(contenu):
    soup = BeautifulSoup(contenu, "html.parser")
    app_div = soup.find('div', {'id': 'app'})

    try:
        if app_div is None:
            return []
        data_page = app_div.attrs.get('data-page') if hasattr(app_div, 'attrs') else None

        if not data_page or not isinstance(data_page, str):
            return []
        
        decoded_data = html.unescape(data_page)
        page_data = json.loads(decoded_data)

        products = []
        recherche_produit(page_data, products)

        return products

    except Exception:
        return []

In [5]:
# Fonction principale pour récupérer la liste des fleurs
def get_products(total_pages: int = 2):
    base_url = "https://fleur-quebec.com/catalog/fleurs/bouquets-melanges"
    flowers_list = []

    for page in range(1, total_pages + 1):
        page_url = f"{base_url}?page={page}"
        print(f"Scraping de la page {page_url}")

        try:
            response = requests.get(page_url, headers=headers, timeout=15)
            if response.status_code != 200:
                print(f"Erreur HTTP {response.status_code} pour la page {page}")
                continue

            products_temp = extrait_contenu_page(response.text)

            if not products_temp:
                print(f"Aucun produit trouvé sur la page {page}")
                break

            flowers_list += products_temp
            print(f"{len(products_temp)} produits trouvés sur la page {page}")

            # Pause pour éviter de surcharger le serveur
            time.sleep(2)

        except Exception as e:
            print(f"Erreur lors du scraping de la page {page}: {e}")
            continue

    return flowers_list

In [6]:
# Exécution du scraping
print("Début du scraping...")

# Récupération des produits (2 pages par défaut)
produits = get_products(2)
print(f"Résultats: {len(produits)} produits extraits au total")


Début du scraping...
Scraping de la page https://fleur-quebec.com/catalog/fleurs/bouquets-melanges?page=1
12 produits trouvés sur la page 1
Scraping de la page https://fleur-quebec.com/catalog/fleurs/bouquets-melanges?page=2
12 produits trouvés sur la page 2
Résultats: 24 produits extraits au total


In [7]:
produits

[{'titre': '12 Roses Orange à Longue Tige',
  'prix_solde': 44.99,
  'prix_avant_reduction': '69.99',
  'note': 5.0,
  'lien_image': 'https://live.data.necs.ca/storage/products/12_roses_orange__longue_tige_thumb.webp?lastmode=1751506976?lastmode=1751506976'},
 {'titre': 'Délice Rose & Rouge',
  'prix_solde': 69.99,
  'prix_avant_reduction': '109.99',
  'note': 5.0,
  'lien_image': 'https://live.data.necs.ca/storage/products/dlice_rose__rouge_thumb.webp?lastmode=1751506976?lastmode=1751506976'},
 {'titre': 'Bouquet Chérie',
  'prix_solde': 44.99,
  'prix_avant_reduction': '74.99',
  'note': 5.0,
  'lien_image': 'https://live.data.necs.ca/storage/products/bouquet_chrie__thumb.webp?lastmode=1751506976?lastmode=1751506976'},
 {'titre': 'Marguerites Ensoleillées',
  'prix_solde': 29.99,
  'prix_avant_reduction': '49.99',
  'note': 5.0,
  'lien_image': 'https://live.data.necs.ca/storage/products/sunshine_daisies_thumb.webp?lastmode=1751506976?lastmode=1751506976'},
 {'titre': 'Meilleur Vende

In [8]:
# Traitement des données et création du DataFrame
df = pd.DataFrame(produits) if produits else pd.DataFrame()

In [9]:
df

,titre,prix_solde,prix_avant_reduction,note,lien_image
0,12 Roses Orange à Longue Tige,44.99,69.99,5.0,https://live.data.necs.ca/storage/products/12_...
1,Délice Rose & Rouge,69.99,109.99,5.0,https://live.data.necs.ca/storage/products/dli...
2,Bouquet Chérie,44.99,74.99,5.0,https://live.data.necs.ca/storage/products/bou...
3,Marguerites Ensoleillées,29.99,49.99,5.0,https://live.data.necs.ca/storage/products/sun...
4,Meilleur Vendeur,90.99,90.99,5.0,https://live.data.necs.ca/storage/products/mei...
5,Ensemble-cadeau de roses d'été et de ballons m...,46.99,46.99,4.0,https://live.data.necs.ca/storage/products/ros...
6,Câlins et Bisous III,39.99,79.99,4.0,https://live.data.necs.ca/storage/products/hug...
7,L'amour le plus pur de Cœur,49.99,69.99,4.0,https://live.data.necs.ca/storage/products/hea...
8,Câlins et Bisous II,34.99,69.99,4.0,https://live.data.necs.ca/storage/products/hug...
9,50 fleurs d'Alstroemeria II,44.99,94.99,5.0,https://live.data.necs.ca/storage/products/50_...


In [10]:
df.head()

,titre,prix_solde,prix_avant_reduction,note,lien_image
0,12 Roses Orange à Longue Tige,44.99,69.99,5.0,https://live.data.necs.ca/storage/products/12_...
1,Délice Rose & Rouge,69.99,109.99,5.0,https://live.data.necs.ca/storage/products/dli...
2,Bouquet Chérie,44.99,74.99,5.0,https://live.data.necs.ca/storage/products/bou...
3,Marguerites Ensoleillées,29.99,49.99,5.0,https://live.data.necs.ca/storage/products/sun...
4,Meilleur Vendeur,90.99,90.99,5.0,https://live.data.necs.ca/storage/products/mei...


In [11]:
# Sauvegarde des données en CSV
if not df.empty:
    # Sauvegardons le résultat dans un fichier CSV
    filename = 'resultat.csv'
    df.to_csv(filename, index=False, encoding='utf-8')

    print(f"Données sauvegardées dans '{filename}'")

    # Affichons le chemin complet du fichier
    full_path = os.path.abspath(filename)
    print(f"Chemin complet : {full_path}")

else:
    print("Aucune donnée à sauvegarder")


Données sauvegardées dans 'resultat.csv'
Chemin complet : /Users/raharison/docs/TP/exam-intra-H2025/resultat.csv
